In [1]:
from DBHandler import Handler
from redis import Redis
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
def initData():
    redis = Redis()
    handler = Handler(redis)
    data = handler.load_all('test')
    return(pd.DataFrame(data))


In [3]:
data = initData()

In [4]:
data

,high,low,open,minute,relative_minute,close
0,1.12770,1.12763,1.12764,23,0,1.12767
1,1.12776,1.12740,1.12767,24,1,1.12751
2,1.12760,1.12730,1.12751,25,2,1.12750
3,1.12758,1.12720,1.12750,26,3,1.12734
4,1.12742,1.12720,1.12734,27,4,1.12726
5,1.12730,1.12700,1.12726,28,5,1.12719
6,1.12730,1.12700,1.12719,29,6,1.12714
7,1.12723,1.12690,1.12714,30,7,1.12709
8,1.12710,1.12679,1.12709,31,8,1.12682
9,1.12690,1.12660,1.12682,32,9,1.12678


In [5]:
scaler = MinMaxScaler()
data[['close', 'high', 'low']] = scaler.fit_transform(data[['close', 'high', 'low']])

In [6]:
data

,high,low,open,minute,relative_minute,close
0,0.948276,1.000000,1.12764,23,0,1.000000
1,1.000000,0.827068,1.12767,24,1,0.863248
2,0.862069,0.751880,1.12751,25,2,0.854701
3,0.844828,0.676692,1.12750,26,3,0.717949
4,0.706897,0.676692,1.12734,27,4,0.649573
5,0.603448,0.526316,1.12726,28,5,0.589744
6,0.603448,0.526316,1.12719,29,6,0.547009
7,0.543103,0.451128,1.12714,30,7,0.504274
8,0.431034,0.368421,1.12709,31,8,0.273504
9,0.258621,0.225564,1.12682,32,9,0.239316


In [7]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [8]:
look_back = 15

In [9]:
time = data['relative_minute']

In [10]:
price = data['close'].values

In [14]:
price = np.reshape(price, (-1, 1))

In [16]:
train = TimeseriesGenerator(price, price, length=look_back, batch_size=10)
print(train.get_config())

{'data': '[[1.0], [0.8632478632479206], [0.8547008547009227], [0.7179487179488433], [0.649572649572633], [0.5897435897435344], [0.5470085470086588], [0.5042735042734421], [0.27350427350427253], [0.2393162393161674], [0.17094017094018454], [0.17094017094018454], [0.17094017094018454], [0.17094017094018454], [0.17094017094018454], [0.11111111111108585], [0.11965811965819739], [0.017094017093995717], [0.03418803418810512], [0.0], [0.0], [0.08547008547009227]]', 'targets': '[[1.0], [0.8632478632479206], [0.8547008547009227], [0.7179487179488433], [0.649572649572633], [0.5897435897435344], [0.5470085470086588], [0.5042735042734421], [0.27350427350427253], [0.2393162393161674], [0.17094017094018454], [0.17094017094018454], [0.17094017094018454], [0.17094017094018454], [0.17094017094018454], [0.11111111111108585], [0.11965811965819739], [0.017094017093995717], [0.03418803418810512], [0.0], [0.0], [0.08547008547009227]]', 'length': 15, 'sampling_rate': 1, 'stride': 1, 'start_index': 15, 'end_i

In [17]:
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train, epochs=num_epochs, verbose=1)

Epoch 1/25
1/1 [==============================] - 1s 912ms/step - loss: 0.0049
Epoch 2/25
1/1 [==============================] - 0s 29ms/step - loss: 0.0046
Epoch 3/25
1/1 [==============================] - 0s 40ms/step - loss: 0.0043
Epoch 4/25
1/1 [==============================] - 0s 16ms/step - loss: 0.0041
Epoch 5/25
1/1 [==============================] - 0s 32ms/step - loss: 0.0038
Epoch 6/25
1/1 [==============================] - 0s 10ms/step - loss: 0.0036
Epoch 7/25
1/1 [==============================] - 0s 15ms/step - loss: 0.0034
Epoch 8/25
1/1 [==============================] - 0s 15ms/step - loss: 0.0032
Epoch 9/25
1/1 [==============================] - 0s 15ms/step - loss: 0.0031
Epoch 10/25
1/1 [==============================] - 0s 14ms/step - loss: 0.0029
Epoch 11/25
1/1 [==============================] - 0s 14ms/step - loss: 0.0028
Epoch 12/25
1/1 [==============================] - 0s 10ms/step - loss: 0.0027
Epoch 13/25
1/1 [==============================] - 0s 10ms/s

In [18]:
num_predict = 5
price = price.reshape(-1)

In [19]:
price

array([1.        , 0.86324786, 0.85470085, 0.71794872, 0.64957265,
       0.58974359, 0.54700855, 0.5042735 , 0.27350427, 0.23931624,
       0.17094017, 0.17094017, 0.17094017, 0.17094017, 0.17094017,
       0.11111111, 0.11965812, 0.01709402, 0.03418803, 0.        ,
       0.        , 0.08547009])

In [21]:
len(price)

22

In [22]:
prediction_list = price[-look_back:]
print(prediction_list)
    

[0.5042735  0.27350427 0.23931624 0.17094017 0.17094017 0.17094017
 0.17094017 0.17094017 0.11111111 0.11965812 0.01709402 0.03418803
 0.         0.         0.08547009]


In [25]:
for _ in range(num_predict):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
prediction_list = prediction_list[look_back-1:]

In [26]:
prediction_list

array([0.08547009, 0.04969755, 0.04889393, 0.04838141, 0.04796845,
       0.0476804 ])